<b>LING 193 - F22<br>
This is the my implementation of the autocorrect system<br></b><br>

Big shout out to my instructor - ***Andrew McInnerney***<br>
He led me through every linguistic concepts I learned in Ling 193.<br>
**"Download dictionary"** and **"1st Layer of Correction: Minimum edit distance"** are modefied based on the code from Andrew.

# Download dictionary (run needed)

In [1]:
# LOAD THE FILES FOR THIS NOTEBOOK
import pathlib
from zipfile import ZipFile

current_path = pathlib.Path.cwd()
file_path = current_path.joinpath("Class 7.zip")
# print(f"check exists: {pathlib.Path.exists(file_path)}")
if not pathlib.Path.exists(file_path):
	!wget -q --show-progress --no-check-certificate 'https://docs.google.com/uc?export=download&id=1cuIbCCZZutqlO-d9s9KxI6GxfbHd8rnb' -O 'Class 7.zip'
	with ZipFile('Class 7.zip', 'r') as zipObj:
		zipObj.extractall()
print("*** Got Class 7.zip")
print("-----------------------------\n")

file_path = current_path.joinpath("stanford-corenlp-4.5.1.zip")
if not pathlib.Path.exists(file_path):
	!wget https://nlp.stanford.edu/software/stanford-corenlp-4.5.1.zip
	with ZipFile('stanford-corenlp-4.5.1.zip', 'r') as zipObj:
		zipObj.extractall()
print("*** Got stanford-corenlp-4.5.1.zip")

Class 7.zip         100%[===================>]   1.04M  --.-KB/s    in 0.008s  
*** Got Class 7.zip
-----------------------------

--2022-12-30 07:41:55--  https://nlp.stanford.edu/software/stanford-corenlp-4.5.1.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-4.5.1.zip [following]
--2022-12-30 07:41:56--  https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-4.5.1.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 505225173 (482M) [application/zip]
Saving to: ‘stanford-corenlp-4.5.1.zip’

stanford-corenlp-4. 100%[===================>] 481.82M  5.09MB/s    in 92s     



# 1st Layer of Correction: Minimum edit distance (run needed)
**minedit()** returns a table with every word in the dictionary listed along with its distance to the target word


> **firstrow()**, **nextrow()**, and **substitution_penalty()** are the helper functions associated with **minedit()** in calculating word distance


**get_mega_candidate()** will find the top 100 candidates with the minimum edit distance to the typo


> ### *How do I index each word*:
> *   get_mega_candidate()   =>   [ (word, min_distance) ... () ... ]
> *   get_mega_candidate()   =>   [ ('THIS', 1) ... () ... ]
> *   get_mega_candidate()[0]   =>   ('THIS', 1)
> *   get_mega_candidate()[0][0]   =>   'THIS'



In [2]:
def firstrow(m):
    return [i for i in range(m+1)]


def substitution_penalty(letter1, letter2):
    if letter1 == letter2:
        return 0
    else:
        return 2


def nextrow(priorrow, word1, letter):
    row = [priorrow[0] + 1]
    priorcell = row[0]
    for i in range(len(word1)):
        insertion = priorrow[i+1] + 1
        deletion = priorcell + 1
        substitution = priorrow[i] + substitution_penalty(word1[i], letter)
        priorcell = min(insertion, deletion, substitution)
        row.append(priorcell)
    return row


def minedit(word1, word2):
    m = len(word1)
    n = len(word2)
    priorrow = firstrow(m)
    table = [priorrow]
    for i in range(n):
        row = nextrow(priorrow, word1, word2[i])
        table.append(row)
        priorrow = row
    return table


# This changes our dictionary to a shorter one of more frequent words
with open("subtlex_words.txt") as file:
    words = file.read().splitlines()

frequencies = {}
for entry in words:
    word = entry.split()[0].upper()
    freq = int(entry.split()[1])
    if word not in frequencies:
        frequencies[word] = freq
dictionary = {word:freq for word,freq in frequencies.items() if freq > 12}


def get_mega_candidate(typo):
    distance = {}
    candidate = []
    for word in dictionary:
        table = minedit(typo,word)
        distance[word] = table[-1][-1]

    for i in range(100):
        temp_word = min(distance, key = distance.get)
        temp_val = distance[temp_word]
        candidate.append((temp_word, temp_val))
        del distance[temp_word]

    return candidate


## Test the 1nd Layer of Autocorrect System: Minimum Edit Distance
Namely, if get_mega_candidate returns a list of candidates

In [3]:
# test correct()
typo = "ani".upper()
print(get_mega_candidate(typo))

[('AN', 1), ('NI', 1), ('AI', 1), ('ANTI', 1), ('A', 2), ('AND', 2), ('I', 2), ('CAN', 2), ('MAN', 2), ('ANY', 2), ('AIN', 2), ('AIR', 2), ('RAN', 2), ('FAN', 2), ('N', 2), ('SAN', 2), ('VAN', 2), ('PANIC', 2), ('AIM', 2), ('AID', 2), ('DAN', 2), ('PAN', 2), ('TAN', 2), ('ANN', 2), ('ANNIE', 2), ('NIP', 2), ('ANT', 2), ('LAN', 2), ('BAN', 2), ('HAN', 2), ('ANA', 2), ('WAN', 2), ('TAI', 2), ('MANIC', 2), ('NIX', 2), ('MANIA', 2), ('AMI', 2), ('NAN', 2), ('ANVIL', 2), ('YAN', 2), ('KAI', 2), ('LAI', 2), ('HANOI', 2), ('NIL', 2), ('UNI', 2), ('BAI', 2), ('TANIA', 2), ('SAI', 2), ('AHI', 2), ('JAI', 2), ('NIG', 2), ('CAI', 2), ('KAN', 2), ('NIM', 2), ('IT', 3), ('IN', 3), ('ON', 3), ('IS', 3), ('NO', 3), ('AT', 3), ('IF', 3), ('WANT', 3), ('AS', 3), ('MEAN', 3), ('SAID', 3), ('AM', 3), ('THAN', 3), ('WAIT', 3), ('NICE', 3), ('WASN', 3), ('MANY', 3), ('HI', 3), ('HAND', 3), ('AREN', 3), ('DAMN', 3), ('HANG', 3), ('HAIR', 3), ('PLAN', 3), ('AH', 3), ('HASN', 3), ('FAIR', 3), ('MA', 3), ('PAI

# 2nd Layer of Correcton: Word Frequency (run needed)
**citformat()** is a helper function that will clean up the punctuation in the given sentence. Its input is each word in the given function. Its output is the cleanned word.

**get_freq_candidate()** will find the correct candidate with the minimum edit distance to the typo. This candidate will also has its word frequency to be the highest among other candidates

**min_distance_correct()** is a driver function that takes in a sentence and return a sentence as the result of 1st layer of correction

In [4]:
def citformat(word):
    punctuation = ["!","?",",",".","-","~",":",";", "'", '"']
    while word[-1] in punctuation:
        word = word[:-1]
    while word[0] in punctuation:
        word = word[1:]
    return word.upper()


def get_freq_candidate(typo):
    errors = {}
    mindist = 100
    for word in dictionary:
        table = minedit(typo,word)
        distance = table[-1][-1]

        if distance < mindist:
          errors = {}
          mindist = distance
        if distance <= mindist:
          errors[word] = distance

    return max(errors, key = dictionary.get)


def min_distance_correct(sentence):
    sentence = sentence.split()
    
    for i in range(len(sentence)):
        sentence[i] = citformat(sentence[i])
        sentence[i] = get_freq_candidate(sentence[i].upper()).lower()
    
    sentence = " ".join(sentence).capitalize()
    return sentence

## Test the 2nd Layer of Autocorrect System: Word Frequency


### Examples:
> s1 = "The manager hired ths employees"\
> s2 = "John will not buy anu tomatoes"


### Min Distance + Frequency Result:
> s1 = "The manager hired **this** employees"\
> s2 = "John will not buy **an** tomatoes"

In [5]:
# Examples
s1 = "The manager hired ths employees."
s2 = "John will not buy anu tomatoes."

s1 = min_distance_correct(s1)
print(s1)

s2 = min_distance_correct(s2)
print(s2)


The manager hired this employees
John will not buy an tomatoes


# Preparing for the 3rd Layer of Correction (run needed)

Install dependencies: nltk package

In [6]:
%pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Find Common "DT" from Brown Corpus

Check each sentences in brwon corpus and find determiners showed in each sentences. 
 
Generate 3 lists: dt, dts, dti
*   dt: singular determiner
*   dts: plural determiner
*   dti: plural quantifier









In [7]:
import nltk
from nltk.corpus import brown
nltk.download('brown')
print("-----------------------------")

dt = []
dts = []
dti = []


browntags = brown.tagged_sents()
for sents in browntags:
    for pair in sents:
        determiner = pair[0].upper()
        if pair[1] == "DT" and determiner not in dt:
            dt.append(determiner)
        elif pair[1] == "DTS" and determiner not in dts:
            dts.append(determiner)
        elif pair[1] == "DTI" and determiner not in dti:
            dti.append(determiner)

print(f"dt: {dt}")
print(f"dts: {dts}")
print(f"dti: {dti}")

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


-----------------------------
dt: ['THIS', 'EACH', 'ANOTHER', 'THAT', "'NOTHER", 'THET']
dts: ['THESE', 'THOSE', 'THEM', 'THEASE', 'THEES']
dti: ['ANY', 'SOME', 'ENNY', 'ANI', 'ANYE']


## Clean the Common "DT" from Brown Corpus

If we narrow our expection for this autocorrect system, we can combine "dt" and "dti" into a single list. In other words, the singular determiner and singular/plural quantifier can be used interchangeably for many scenarios.

This design would work out with our examples, sentence s1 & sentence s2. However, it also indicates a direction for further improvement for this autocorrect system. 

In [8]:
# Clean dt
# Remove informal/colloquial singular determiners
try:
    dt.remove("'NOTHER")
    dt.remove("THET")
    dt.append("THE")
except:
    print("dt is alreay cleanned")

dt.sort(key=dictionary.get, reverse=True) # Sort dt with word frequency
print(f"cleanned dt: {dt}\n")

# Clean dts
# Remove informal/colloquial plural determiners
try:
    dts.remove("THEASE")
    dts.remove("THEES")
    dts.remove("THEM")
    dts.append("THE")
except:
    print("dts is alreay cleanned")

dts.sort(key=dictionary.get, reverse=True) # Sort dts with word frequency
print(f"cleanned dts: {dts}\n")

# Clean dti
# Remove informal/colloquial plural quantifier
try:
    dti.remove("ENNY")
    dti.remove("ANI")
    dti.remove("ANYE")
    dti.append("THE")
except:
    print("dti is alreay cleanned")

dti.sort(key=dictionary.get, reverse=True) # Sort dti with word frequency
print(f"cleanned dti: {dti}\n")

print("---------------------------------------------------")
# Merge dts and dti
dt = dt + dti;
dt = sorted(list(set(dt)), key=dictionary.get, reverse=True)
print(f"merged dt: {dt}")

cleanned dt: ['THE', 'THAT', 'THIS', 'ANOTHER', 'EACH']

cleanned dts: ['THE', 'THESE', 'THOSE']

cleanned dti: ['THE', 'SOME', 'ANY']

---------------------------------------------------
merged dt: ['THE', 'THAT', 'THIS', 'SOME', 'ANY', 'ANOTHER', 'EACH']


# 3rd Layer of Correction: Determiner & Noun Phrase (run needed)

**analyze_tag()** is a helper function that takes in a sentence and returns a the tag for each words in the given sentence

**find_determiner()** finds the correct determiner in the candidates and matches it with the determiners' set

***determiner_correct()*** is the driver function for furter correction. It anlyzes each determiner and the noun phrase after it. If it finds a mismatch, it will change the form of the determiner to match the noun phrase.

In [9]:
# Importing into Python
import nltk
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')


def analyze_tag(sentence):
    word_tag = nltk.pos_tag(word_tokenize(sentence))
    return word_tag


def find_determiner(candidates, determiner):

    for idx in range(len(candidates)):
        correct_word = candidates[idx][0]

        if correct_word in determiner:
            break

    return correct_word


def determiner_correct(word_tag):
    result = []
    for idx in range(len(word_tag)):
        current_word = word_tag[idx][0]
        current_tag = word_tag[idx][1]

        if idx == len(word_tag) - 1:
            # print("----------Last word----------")
            result.append(current_word)
            # print(f"add {current_word}")
            break

        next_word = word_tag[idx + 1][0]
        next_tag = word_tag[idx + 1][1]

        candidates = get_mega_candidate(current_word.upper())
        if current_tag == "DT" and next_tag == "NNS":
            # print("----------NNS----------")
            # print(f"NNS candidate: {candidates}")

            result.append(find_determiner(candidates, dts))
            # print(f"add {find_determiner(candidates, dts)}")
            
        elif current_tag == "DT" and next_tag == "NN":
            # print("----------NN----------")
            # print(f"NN candidate: {candidates}")

            result.append(find_determiner(candidates, dt))
            # print(f"add {find_determiner(candidates, dt)}")
    
        else:
            # print("----------No Change----------")
            result.append(current_word)
            # print(f"add {current_word}")

    return " ".join(result).capitalize()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Test Analyze Tags for the given examples, s1 & s2

In [10]:
# Importing into Python
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Original sentences:
# s1 = "The manager hired ths employees."
# s2 = "John will not buy an tomatoes."

# After the 1st layer of autocorrection:
s1 = "The manager hired this employees"
s2 = "John will not buy an tomatoes"
test = "ane anu anr so"

print(analyze_tag(s1), "\n")
print(analyze_tag(s2), "\n")
print(analyze_tag(test), "\n")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('The', 'DT'), ('manager', 'NN'), ('hired', 'VBD'), ('this', 'DT'), ('employees', 'NNS')] 

[('John', 'NNP'), ('will', 'MD'), ('not', 'RB'), ('buy', 'VB'), ('an', 'DT'), ('tomatoes', 'NN')] 

[('ane', 'NN'), ('anu', 'NN'), ('anr', 'NN'), ('so', 'RB')] 



# Driver function: Combine all function pieces (run needed)

### Examples:
> s1 = "The manager hired ths employees"\
> s2 = "John will not buy anu tomatoes"


### Min Distance + Frequency Result:
> s1 = "The manager hired **this** employees"\
> s2 = "John will not buy **an** tomatoes"

### Determiner & Noun Phrase Result:
> s1 = "The manager hired **the** employees"\
> s2 = "John will not buy **any** tomatoes"

In [11]:
s1 = "The manager hired thi employees."
s2 = "John will not buy anu tomatoes."

def autocorrect(sentence):
    print(f"*** Original sentence: {sentence}")

    sentence = min_distance_correct(sentence).capitalize()
    print("*** Min distance + Frequency result: ", sentence)

    word_tag = analyze_tag(sentence)
    # print("Tag:", word_tag)

    result = determiner_correct(word_tag)
    # print("Final result: ", result, "\n")

    return result

print(f"*** Final result: {autocorrect(s1)}\n")
print(f"*** Final result: {autocorrect(s2)}")


*** Original sentence: The manager hired thi employees.
*** Min distance + Frequency result:  The manager hired this employees
*** Final result: The manager hired the employees

*** Original sentence: John will not buy anu tomatoes.
*** Min distance + Frequency result:  John will not buy an tomatoes
*** Final result: John will not buy any tomatoes


# Examples:
### Sample:
It is widelly taugt that parts of speech are definned in terms of sinple defimitions. For example, "a noun is a person place or thinge." But in
reality, simple defintions like that are not very usefful. For exampe, 
is it not true that everthing is a "thing"? Then in wghat sense can we
deffine nouns as "thngs"? We need moree rigorouss teckneques to define
these notioms.


### Result:
It is widely taught that parts of speech are defined in terms of simple definitions for example any noun is any person place or thing but in reality simple definitions like that are not very useful for example is it not true that everything is any thing then in what sense can we define nouns as things we need more rigorous techniques to define these notions

In [12]:
sample = """It is widelly taugt that parts of speech are definned in terms of sinple
defimitions. For example, "a noun is a person place or thinge." But in
reality, simple defintions like that are not very usefful. For exampe, 
is it not true that everthing is a "thing"? Then in wghat sense can we
deffine nouns as "thngs"? We need moree rigorouss teckneques to define
these notioms."""


print(f"*** Final result: {autocorrect(sample)}\n")


*** Original sentence: It is widelly taugt that parts of speech are definned in terms of sinple
defimitions. For example, "a noun is a person place or thinge." But in
reality, simple defintions like that are not very usefful. For exampe, 
is it not true that everthing is a "thing"? Then in wghat sense can we
deffine nouns as "thngs"? We need moree rigorouss teckneques to define
these notioms.
*** Min distance + Frequency result:  It is widely taught that parts of speech are defined in terms of simple definitions for example a noun is a person place or thing but in reality simple definitions like that are not very useful for example is it not true that everything is a thing then in what sense can we define nouns as things we need more rigorous techniques to define these notions
*** Final result: It is widely taught that parts of speech are defined in terms of simple definitions for example any noun is any person place or thing but in reality simple definitions like that are not very use